## INK: explanatory notebook

Within this notebook, we give a simple example of how the INK library can be used to extract the neighbourhood of certain subjects of interest.<br>
We also show the mining capabilities for both task specific and task agnostic cases.


The example dataset can be found in the /datasets folder.<br>
We have used the common known animal dataset, describing several animals with their properties.

In [73]:
from ink.base.connectors import RDFLibConnector
from ink.base.structure import InkExtractor
from ink.miner.rulemining import RuleSetMiner

To start, three different packages are loaded.
* A connector: which is used to load the original dataset.
Here we have used an RDFLib connector, but other connectors are available.
* The INK extractor, which will transform the neighbourhood of certain nodes into a binary representation.
* The Rule Set Miner, which is our rule mining module.

In [74]:
import pandas as pd
import numpy as np

customer = pd.read_csv('train.tsv', sep='\t')
cus_seg = pd.read_csv('train.csv')

In [75]:
customer.head(8)

,subject,predicate,object
0,Customer15634602,Gender,GenderFemale
1,Customer15634602,Age,Age42
2,Customer15634602,Geography,GeographyFrance
3,Customer15634602,Tenure,Tenure2
4,Customer15634602,NumOfProducts,NumOfProducts1
5,Customer15634602,HasCrCard,HasCrCardYes
6,Customer15634602,IsActiveMember,IsActiveMemberYes
7,Customer15647311,Gender,GenderFemale


In [76]:
cus_seg.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [77]:
cus_sub = customer['subject'].unique() #row #10000
cus_sub

array(['Customer15634602', 'Customer15647311', 'Customer15619304', ...,
       'Customer15584532', 'Customer15682355', 'Customer15628319'],
      dtype=object)

In [78]:
cus_pre = customer['predicate'].unique() #7
cus_pre

array(['Gender', 'Age', 'Geography', 'Tenure', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember'], dtype=object)

In [79]:
cus_obj = customer['object'].unique() #94
cus_obj

array(['GenderFemale', 'Age42', 'GeographyFrance', 'Tenure2',
       'NumOfProducts1', 'HasCrCardYes', 'IsActiveMemberYes', 'Age41',
       'GeographySpain', 'Tenure1', 'HasCrCardNo', 'Tenure8',
       'NumOfProducts3', 'IsActiveMemberNo', 'Age39', 'NumOfProducts2',
       'Age43', 'GenderMale', 'Age44', 'Age50', 'Tenure7', 'Age29',
       'GeographyGermany', 'Tenure4', 'NumOfProducts4', 'Age27', 'Age31',
       'Tenure6', 'Age24', 'Tenure3', 'Age34', 'Tenure10', 'Age25',
       'Tenure5', 'Age35', 'Age45', 'Age58', 'Tenure9', 'Age32', 'Age38',
       'Age46', 'Age36', 'Tenure0', 'Age33', 'Age40', 'Age51', 'Age61',
       'Age49', 'Age37', 'Age19', 'Age66', 'Age56', 'Age26', 'Age21',
       'Age55', 'Age75', 'Age22', 'Age30', 'Age28', 'Age65', 'Age48',
       'Age52', 'Age57', 'Age73', 'Age47', 'Age54', 'Age72', 'Age20',
       'Age67', 'Age79', 'Age62', 'Age53', 'Age80', 'Age59', 'Age68',
       'Age23', 'Age60', 'Age70', 'Age63', 'Age64', 'Age18', 'Age82',
       'Age69', 'Age74', 'A

###### rule miner (不知道可不可以用(((o(*ﾟ▽ﾟ*)o))))

In [80]:
miner = RuleSetMiner(chains=100, max_len_rule_set=3, forest_size=10)

###### task specific rule mining

In [81]:
df = pd.crosstab(customer['subject'], customer['object']) #add .ne(0) to transform 0,1 to True False 

In [82]:
df.head(2)

object,Age18,Age19,Age20,Age21,Age22,Age23,Age24,Age25,Age26,Age27,...,Tenure1,Tenure10,Tenure2,Tenure3,Tenure4,Tenure5,Tenure6,Tenure7,Tenure8,Tenure9
subject,,,,,,,,,,,,,,,,,,,,,
Customer15565701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Customer15565706,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [83]:
#df.filter(regex='^Age',axis=1).head()

In [84]:
df.columns = ['rdf#Gender§' + str(col) if col in df.filter(regex='^Gender',axis=1) else col for col in df.columns]
df.columns = ['rdf#Age§' + str(col) if col in df.filter(regex='^Age',axis=1) else col for col in df.columns]
df.columns = ['rdf#Geography§' + str(col) if col in df.filter(regex='^Geography',axis=1) else col for col in df.columns]
df.columns = ['rdf#Tenure§' + str(col) if col in df.filter(regex='^Tenure',axis=1) else col for col in df.columns]
df.columns = ['rdf#NumOfProducts§' + str(col) if col in df.filter(regex='^NumOfProducts',axis=1) else col for col in df.columns]
df.columns = ['rdf#HasCrCard§' + str(col) if col in df.filter(regex='^HasCrCard',axis=1) else col for col in df.columns]
df.columns = ['rdf#IsActiveMember§' + str(col) if col in df.filter(regex='^IsActiveMember',axis=1) else col for col in df.columns]

In [85]:
df.head(10)

,rdf#Age§Age18,rdf#Age§Age19,rdf#Age§Age20,rdf#Age§Age21,rdf#Age§Age22,rdf#Age§Age23,rdf#Age§Age24,rdf#Age§Age25,rdf#Age§Age26,rdf#Age§Age27,...,rdf#Tenure§Tenure1,rdf#Tenure§Tenure10,rdf#Tenure§Tenure2,rdf#Tenure§Tenure3,rdf#Tenure§Tenure4,rdf#Tenure§Tenure5,rdf#Tenure§Tenure6,rdf#Tenure§Tenure7,rdf#Tenure§Tenure8,rdf#Tenure§Tenure9
subject,,,,,,,,,,,,,,,,,,,,,
Customer15565701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Customer15565706,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Customer15565714,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Customer15565779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Customer15565796,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
Customer15565806,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Customer15565878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Customer15565879,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Customer15565891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, Customer15565701 to Customer15815690
Data columns (total 94 columns):
 #   Column                                Non-Null Count  Dtype
---  ------                                --------------  -----
 0   rdf#Age§Age18                         10000 non-null  int64
 1   rdf#Age§Age19                         10000 non-null  int64
 2   rdf#Age§Age20                         10000 non-null  int64
 3   rdf#Age§Age21                         10000 non-null  int64
 4   rdf#Age§Age22                         10000 non-null  int64
 5   rdf#Age§Age23                         10000 non-null  int64
 6   rdf#Age§Age24                         10000 non-null  int64
 7   rdf#Age§Age25                         10000 non-null  int64
 8   rdf#Age§Age26                         10000 non-null  int64
 9   rdf#Age§Age27                         10000 non-null  int64
 10  rdf#Age§Age28                         10000 non-null  int64
 11  rdf#Age§Age29       

In [87]:
from scipy.sparse import csr_matrix  #convert df to csr
df_csr = csr_matrix(df.values)
df_csr

<10000x94 sparse matrix of type '<class 'numpy.int64'>'
	with 70000 stored elements in Compressed Sparse Row format>

In [88]:
col = list(df.columns)
#col

In [89]:
cus_seg_sort = cus_seg.sort_values(by=['CustomerId'])
#cus_seg_sort

In [90]:
label = cus_seg_sort['Exited']
y_train = np.array(label)  #y_train #label #10000
y_train

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

In [94]:
X_train = (df_csr,list(cus_sub),col)
#X_train

In [97]:
#X_train: data
#y_train: label
acc, rules = miner.fit(X_train, y_train)

c:\Users\OWNER\Desktop\INK\INK\ink\miner\rulemining.py:223: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.rules = np.asarray(self.rules)[supp_select]


In [98]:
print(acc)
miner.print_rules(rules)

0.7944
['rdf#NumOfProducts§NumOfProducts3']
['rdf#NumOfProducts§NumOfProducts1', 'rdf#Geography§GeographyGermany']
